In [1]:
# CELL #1
import matplotlib
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from matplotlib.widgets import Slider, Button, RadioButtons
import matplotlib.gridspec as gridspec

%matplotlib tk

import tkinter as tk
from tkinter import filedialog

import matplotlib.cm as cm
from matplotlib import gridspec
from scipy import signal

import numpy as np
import os
#import shutil
import cv2

#import glob2

#import tables
#import scipy
import csv

#import dependency functions

# from utility import (fix_nans, 
#                      del_singles,
#                      del_triples,
#                      del_segment, 
#                      interpolate_traces,
#                      update,
#                      animal_select,
#                      animal_switch_function,
#                      next_segment,
#                      forward_function,
#                      backward_function,
#                      del_jumps_function,
#                      previous_segment,
#                      save,
#                      load_function,
#                      interpolate_function,
#                      export_movie                     
# )


Autosaving every 180 seconds


In [2]:
# CELL #2

def clean_data(event):
    global tracesx, tracesy

    ids = np.arange(4)
    for id_ in ids:
        for k in range(tracesx.shape[0]):
            if np.isnan(tracesx[k,id_])!=np.isnan(tracesy[k,id_]):
                print (k, tracesx[k,id_], tracesy[k,id_])
                tracesx[k,id_]= tracesy[k,id_] =np.nan
                print ('')
                
    print ("DONE matching y and x NANS")
    
    # run delete values on the boundaries
    for id_ in ids:
        idx = np.where(tracesx[:,id_]==0)
        tracesx[idx,id_]=np.nan
        
        idx = np.where(tracesx[:,id_]==1279)
        tracesx[idx,id_]=np.nan
        
        idx = np.where(tracesy[:,id_]==0)
        tracesy[idx,id_]=np.nan

        idx = np.where(tracesy[:,id_]==1023)
        tracesy[idx,id_]=np.nan


    print ("DONe deleting traces on boundaries")

    # delete short segments 
    del_short_segments(0)

    print ("DONE Deleting short segments")
    
    update(current_index[0])
    
def del_singles(event):
    global tracesx, tracesy
    
    # check to see in any sequence of 3 if the middle si the only real value
    ids = np.arange(4)
    for id_ in ids:
        for k in range(0,tracesx.shape[0]-2,1):
            if np.all(np.isnan(tracesx[k:k+3,id_])==np.array([True, 
                                                 False,
                                                 True])):
                tracesx[k:k+3,id_]=np.nan
                tracesy[k:k+3,id_]=np.nan
    print ("DONE DELEting singles")   
    update(current_index[0])
    
    
def del_short_segments(event):
    global tracesx, tracesy
    
    # check to see in any sequence of 3 if the middle si the only real value
    ids = np.arange(4)
    for id_ in ids:
        for k in range(0,tracesx.shape[0]-4,1):
            flag_ = False
            if np.all(np.isnan(tracesx[k:k+5,id_])==np.array([True,
                                                              False,False,False,
                                                              True])):
                tracesx[k:k+5,id_]=np.nan
                tracesy[k:k+5,id_]=np.nan
                flag_ = True
                
            elif np.all(np.isnan(tracesx[k:k+4,id_])==np.array([True,
                                                              False,False,
                                                              True])):
                tracesx[k:k+4,id_]=np.nan
                tracesy[k:k+4,id_]=np.nan
                flag_ = True
                
            elif np.all(np.isnan(tracesx[k:k+3,id_])==np.array([True,
                                                              False,
                                                              True])):
                tracesx[k:k+3,id_]=np.nan
                tracesy[k:k+3,id_]=np.nan 
                flag_ = True
                
            if flag_:
                print ("deleted small segs", id_, k)
    print ("DONE DELEting triples")   
    update(current_index[0])

def find_continous_snippet(current_index, tracesx, id_):
    end=n_frames-1
    for k in range(current_index[0],n_frames,1):
        if np.isnan(tracesx[k,id_]):
            end = k
            break

    # search backward
    start=0
    for k in range(current_index[0],0,-1):
        if np.isnan(tracesx[k,id_]):
            start = k+1
            break

    return end, start

def del_segment(event):
    global tracesx, tracesy, autoadvance
    
    # search active trace snippet
    # search forward


 
    #end2, start2 = find_continous_snippet(current_index, tracesx, switch_id)
    end1, start1 = find_continous_snippet(current_index, tracesx, selected_animal[0])

    print ("Selected animal id: ", selected_animal, ", start/end: ", start1, end1)

    # replace switch id info
    tracesx[start1:end1, selected_animal[0]] = np.nan
    tracesy[start1:end1, selected_animal[0]] = np.nan
  
    # move to the next snippet
    if autoadvance:
        print ("audo advance: ", autoadvance)
        next_segment(0)
        
        
    print ("DONE DELEting triples")   
    update(current_index[0])
    
    
    
 

def disconnect_jumps():
    global tracesx, tracesy
    
    #tracesx_ip=tracesx.copy()
    #tracesy_ip=tracesy.copy()
    for k in range(tracesx.shape[1]):

        # for big jumps, set intermediate values to nans
        max_jump = 50
        for p in range(0,tracesx.shape[0]-1,1):
            distx = tracesx[p,k]-tracesx[p+1,k]
            disty = tracesy[p,k]-tracesy[p+1,k]
            dist = np.sqrt(distx**2+disty**2)
            if dist>max_jump:
                #print (snippets[p,0],snippets[p,1])
                tracesx[p:p+2,k]= np.nan
                tracesy[p:p+2,k]= np.nan
                #print ("found large jump: ", p/25.)
            #else:
            #    print (dist, "not merging")


       # break
        #np.isnan(x_loc) or np.isnan(y_loc):
    print ("DONE disconnecting JUMPS")
    #print ("tracex: ", tracesx[:100,0])
    
    return tracesx, tracesy


# def disconnect_jumps():
#     global tracesx, tracesy
    
#     #tracesx_ip=tracesx.copy()
#     #tracesy_ip=tracesy.copy()
#     for k in range(tracesx.shape[1]):

#         # for big jumps, set intermediate values to nans
#         max_jump = 50
#         for p in range(0,tracesx.shape[0]-1,1):
#             distx = tracesx[p,k]-tracesx[p+1,k]
#             disty = tracesy[p,k]-tracesy[p+1,k]
#             dist = np.sqrt(distx**2+disty**2)
#             if dist>max_jump:
#                 #print (snippets[p,0],snippets[p,1])
#                 tracesx[p:p+2,k]= np.nan
#                 tracesy[p:p+2,k]= np.nan
#                 #print ("found large jump: ", p/25.)
#             #else:
#             #    print (dist, "not merging")


#        # break
#         #np.isnan(x_loc) or np.isnan(y_loc):
#     print ("DONE disconnecting JUMPS")
#     #print ("tracex: ", tracesx[:100,0])
    
#     return tracesx, tracesy



# def interpolate_traces_dynamic(tracesx, tracesy, dist_threshold=10., time_threshold=10):
#     tracesx_ip=tracesx.copy()
#     tracesy_ip=tracesy.copy()

#     # 
#     k = selected_animal[0]
    
    
#     # find continous snippets
#     start = False
#     end = False

#     def find_all_continous_snippets(tracesx_temp, tracesy_temp):
        
#         idx = np.where(~np.isnan(tracesx_temp))[0]

#         # make snippets
#         snippets=[]
#         locs = []
#         start = idx[0]
#         #for p in range(1,idx.shape[0],1):
#         p=1
#         ctr=0
#         while True:
#             if idx[p]-idx[p-1]>1:
#                 end = idx[p-1]
#                 snippets.append([start,end])
#                 start = idx[p]   

#                 # find locations of start and end
#                 locs.append([])
#                 locs[ctr].append([tracesx_temp[start], tracesy_temp[start]])
#                 locs[ctr].append([tracesx_temp[end], tracesy_temp[end]])

#                 # bump 2 over
#                 p+=2
#                 ctr+=1
#             else:
#                 p+=1

#             if p>=idx.shape[0]:
#                 break

#         snippets = np.array(snippets)
#         locs = np.array(locs)

#         return snippets, locs
# #     print (locs.shape)
# #     print (locs[0])
#     snippets, locs = find_all_continous_snippets(tracesx[:,k], tracesy[:,k])
   

#     # loop over all snippets and join them if
#     #  they are lower than 2 x threshold +
#     #  there is nothing within 2x threshold nearby
     
#     for p in range(1, snippets.shape[0]-1):
#         # Skip if interpolate across too long a sgement
#         if (snippets[p,0] - snippets[p-1,1])>2*time_threshold:
#             print ("skipping snippet: ",p ,  " too much time ")
#             continue
        
#         len_snippet = snippets[p,0]- snippets[p-1,1]
        
#         dist = np.linalg.norm(locs[p,0]-locs[p-1,1])

#         #print ("Dist; ", dist, locs[p,1],locs[p+1,0])
#         if dist<2*dist_threshold:
            
#             link = True
#             # check to see if any of the other traces are nearby
#             for l in range(4):
#                 if l == k:
#                     continue
                    
#                 # grab snippets
#                 tempx = tracesx_ip[snippets[p-1,1]:snippets[p,0],l]
#                 tempy = tracesy_ip[snippets[p-1,1]:snippets[p,0],l]
#                 loc_temp = np.vstack((tempx,tempy)).T
#                 print ("snippet: ", snippets[p-1,1], snippets[p,0])
#                 print ("LOcations of animal ", l, "  ", loc_temp.shape)
#                 # compute minimum distance between the other animal points and the start point
                
#                 dist_multi = np.linalg.norm(loc_temp-locs[p-1,1])
#                 print ("dist multi: ", dist_multi)
#                 if np.min(dist_multi)<(2*dist_threshold):
#                     link = False
#                     break
            
#             # if linking then 
#             if link:
#                 # option 1 interpolate and overwrite; THIS IS NOT WORKING WELL
#                 if False:
#                     x_old = np.array([tracesx_ip[snippets[p-1,1], k], 
#                                       tracesx_ip[snippets[p,0], k]])
#                     y_old = np.array([tracesy_ip[snippets[p-1,1], k], 
#                                       tracesy_ip[snippets[p,0], k]])
#                     print ("xodl:", x_old)
#                     print ("yold:", y_old)
#                     x_new = np.linspace(x_old[0], x_old[-1],len_snippet)
#                     print ("x_new: ", x_new)
#                     y_new = np.interp(x_new, x_old, y_old)

#                     print ("interpoalting snippets: ", k, snippets[p,0],snippets[p,1])
#                     tracesx_ip[snippets[p-1,1]:snippets[p,0],k]= x_new
#                     tracesy_ip[snippets[p-1,1]:snippets[p,0],k]= y_new
                
#                 # just switch the ID of the next snippet to the same as previous:
#                 else:
#                     snippets, locs = find_snippets(tracesx[:,k], tracesy[:,k])

                    
                    
#     update(current_index[0])

#     return tracesx_ip, tracesy_ip

       
def find_all_continous_snippets(tracesx_temp, tracesy_temp):
        
    idx = np.where(~np.isnan(tracesx_temp))[0]

    # make snippets
    snippets=[]
    locs = []
    start = idx[0]
    #for p in range(1,idx.shape[0],1):
    p=1
    ctr=0
    while True:
        if idx[p]-idx[p-1]>1:
            end = idx[p-1]
            snippets.append([start,end])
            start = idx[p]   

            # find locations of start and end
            locs.append([])
            locs[ctr].append([tracesx_temp[start], tracesy_temp[start]])
            locs[ctr].append([tracesx_temp[end], tracesy_temp[end]])

            # bump 2 over
            p+=2
            ctr+=1
        else:
            p+=1

        if p>=idx.shape[0]:
            break

    snippets = np.array(snippets)
    locs = np.array(locs)

    return snippets, locs


# TODO Add time threshold also; 
def interpolate_traces(dist_threshold=10., time_threshold=10):
    
    global tracesx, tracesy
    
    for k in range(tracesx.shape[1]):

        snippets, locs = find_all_continous_snippets(tracesx[:,k], tracesy[:,k])

        # fill in space between snippets if euclidean distance < threshold
        #threshold = 10.
        # AND IF NOT TOO FAR APART; 
        for p in range(1, snippets.shape[0]-1):
            # Skip if interpolate across too long a sgement
            if (snippets[p,0] - snippets[p-1,1])>time_threshold:
                continue
            dist = np.linalg.norm(locs[p,0]-locs[p-1,1])
            
            #print ("Dist; ", dist, locs[p,1],locs[p+1,0])
            if dist<dist_threshold:
                #print ("interpoalting snippets: ", k, snippets[p,0],snippets[p,1])
                tracesx[snippets[p-1,1]:snippets[p,0]+1,k]= tracesx[snippets[p-1,1],k]
                tracesy[snippets[p-1,1]:snippets[p,0]+1,k]= tracesy[snippets[p-1,1],k]
            #else:
            #    print (dist, "not merging")

    # 
    print ("DELETING JUMPS FUCNTION")
    #tracesx, tracesy = disconnect_jumps(tracesx,tracesx)

    update(current_index[0])
    #fig.canvas.draw_idle()
    
    return tracesx, tracesy


# update the slider
def update(val):
    global tracesx, tracesy, sizes, img, original_vid, axes_animals, ax_tracesx, ax_tracesy, fig

    #amp = samp.val
    if int(sfreq.val)!=current_index[0]:
        index = int(sfreq.val)
        current_index[0] = index
    else:
        index = current_index[0]
    print ("current indx updated:", current_index)
    
    # grab frame and update it
    original_vid.set(cv2.CAP_PROP_POS_FRAMES, index)
    ret, frame = original_vid.read()
    img.set_data(frame)

    # update scatter plots
    for k in range(4):
        x_loc = tracesx[index,k]
        y_loc = tracesy[index,k]
        
        # If no location inferred, freeze and turn off labels
        if np.isnan(x_loc) or np.isnan(y_loc):
            axes_animals[k].set_alpha(0.0)
            continue
        else:
            axes_animals[k].set_alpha(1.0)
            axes_animals[k].set_sizes([sizes[k]])
            axes_animals[k].set_offsets(np.c_[x_loc,y_loc])
    
    # update trace plots
    ax_tracesx.clear()
    ax_tracesx.set_ylim(0,1280)

    ax_tracesy.clear()
    ax_tracesy.set_ylim(0,1024)
    for k in range(4):
        ax_tracesx.plot(t+current_index[0]/25., tracesx[index:index+init_length,k],
                        linewidth=sizes[k]//50,
                       c=clrs[k])
        #ax1[k].set_linewidth(sizes[k]//50)
        #ax2[k].set_ydata(tracesy[index:index+init_length,k])    
        #ax2[k].set_linewidth(sizes[k]//50)
        ax_tracesy.plot(t+current_index[0]/25., tracesy[index:index+init_length,k], 
                        linewidth=sizes[k]//50,
                       c=clrs[k])
        
    #print ("t value: ", t)
    #print ("second t: ", t+current_index[0]/25.)
    fig.canvas.draw_idle()
    
def advance_function(label):
    global autoadvance
    print ("label: ", label)
    if label == "ON":
        autoadvance=1
    else:
        autoadvance=0
        
    print ("Auto advance: ", autoadvance)
 
def multi_animal_skip(label):
    global multianimal
    print ("label: ", label)
    if label == "ON":
        multianimal=1
    else:
        multianimal=0
        
    print ("multianimal: ", multianimal)    


def animal_select(label):
    #l.set_color(label)
    #fig.canvas.draw_idle()
    
    sizes[:] = 50
    for k in range(4):
        if labels[k] == label:
            break
    sizes[k]=250
    selected_animal[0]=k
    update(current_index[0])

def switch_all_screen(event):    

    global tracesx, tracesy
    print ('')
    print ('')
    print ("SWITCH ALL TRACES ON SCREEN BETWEEN 2 ANIMALS")

    label = animal_switch.value_selected
    print ("label: ", label)
    for k in range(4):
        if labels[k] == label:
            break
    switch_id = k    
    
    print ("curent animal: ", selected_animal[0], " switch animal = ", k )
    
    start = current_index[0]
    end = current_index[0]*10*25-1  # swap all 10 seconds visible on screen
    
    tempx = tracesx.copy()
    tempy = tracesy.copy()
    
    # replace switch id info
    tracesx[start:end, switch_id] = tempx[start:end, selected_animal[0]].copy()
    tracesy[start:end, switch_id] = tempy[start:end, selected_animal[0]].copy()
    
    # replace currental animal info
    tracesx[start:end, selected_animal[0]] = tempx[start:end, switch_id].copy()
    tracesy[start:end, selected_animal[0]] = tempy[start:end, switch_id].copy()
    
    # update plots
    update(current_index[0])   
    
    
def animal_swap(label):
    #l.set_color(label)
    global tracesx, tracesy
    print ('')
    print ('')
    print ("SWITCH ALL TRACES BETWEEN 2 ANIMALS FUNCTION")

    label = animal_switch.value_selected
    print ("label: ", label)
    for k in range(4):
        if labels[k] == label:
            break
    switch_id = k    
    
    print ("curent animal: ", selected_animal[0], " switch animal = ", k )
    
    tempx = tracesx.copy()
    tempy = tracesy.copy()
    
    # replace switch id info
    tracesx[:, switch_id] = tempx[:, selected_animal[0]].copy()
    tracesy[:, switch_id] = tempy[:, selected_animal[0]].copy()
    
    # replace currental animal info
    tracesx[:, selected_animal[0]] = tempx[:, switch_id].copy()
    tracesy[:, selected_animal[0]] = tempy[:, switch_id].copy()
    
    # update plots
    update(current_index[0])   
 
def animal_switch_function(label):
    #l.set_color(label)
    
    global tracesx, tracesy, autoadvance
    print ('')
    print ('')
    print ("SWITCH FUNCTION")

    for k in range(4):
        if labels[k] == label:
            break
    switch_id = k


    # search active trace snippet
    # search forward

    def find_continous_snippet(current_index, tracesx, id_):
        end=n_frames-1
        for k in range(current_index[0],n_frames,1):
            if np.isnan(tracesx[k,id_]):
                end = k
                break

        # search backward
        start=0
        for k in range(current_index[0],0,-1):
            if np.isnan(tracesx[k,id_]):
                start = k+1
                break
    
        return end, start
    
    print ("switching to : ", labels[switch_id])
    print ("current index inside switch:", current_index)
    
    end2, start2 = find_continous_snippet(current_index, tracesx, switch_id)
            
    print ("Switch animal id: ", switch_id, ", start/end: ", start2, end2)
    
    end1, start1 = find_continous_snippet(current_index, tracesx, selected_animal[0])
    
    print ("selected animal id: ", selected_animal[0], ", start/end: ", start1, end1)
        
    
    if end2>end1:
        end1=end2
    
    # DATA SWITCH HAS TO BE SAFE FOR ALL DOWNSTREAM 
    # SO NEED TO CHECK THE ID THAT"S BEING SWAPPED TO MAKE SURE IT DOESN"T GET DELETED;
    # switch the data
    tempx = tracesx.copy()
    tempy = tracesy.copy()
    
#     print ("Target switch tracesx: ", tempx[start1:end1, switch_id].copy())
#     print ("Selected animal tracesx: ", tempx[start1:end1, selected_animal[0]].copy())

#     buff = 2
#     print ("Switch ANIMAL TRACES preswitch; animal: ", switch_id,
#           tracesx[start1-buff:end1+buff, switch_id],
#           tracesy[start1-buff:end1+buff, switch_id])
        

    # replace switch id info
    tracesx[start1:end1, switch_id] = tempx[start1:end1, selected_animal[0]].copy()
    tracesy[start1:end1, switch_id] = tempy[start1:end1, selected_animal[0]].copy()
    
#     print ("Switch ANIMAL TRACES postswitch; animal: ", switch_id, 
#           tracesx[start1-buff:end1+buff, switch_id],
#           tracesy[start1-buff:end1+buff, switch_id])
           
#     print ("Selected ANIMAL TRACES preswitch; animal: ",selected_animal[0],
#           tracesx[start1-buff:end1+buff, selected_animal[0]],
#           tracesy[start1-buff:end1+buff, selected_animal[0]])

    # replace currental animal info
    tracesx[start1:end1, selected_animal[0]] = tempx[start1:end1, switch_id].copy()
    tracesy[start1:end1, selected_animal[0]] = tempy[start1:end1, switch_id].copy()
    
#     print ("SelectedANIMAL TRACES postswitch; animal ",selected_animal[0],
#           tracesx[start1-buff:end1+buff, selected_animal[0]],
#           tracesy[start1-buff:end1+buff, selected_animal[0]])
  
    # 
    #print ("DELETING JUMPS FUCNTION")
    #disconnect_jumps()

    # move to the next snippet
    if autoadvance:
        print ("audo advance: ", autoadvance)
        next_segment(0)
        
    update(current_index[0])   
    

# search for the next continuous chunk of red
def next_segment(event):

    global multianimal, selected_animal, current_index, sizes
    
    # 
    # find the next snippet for the currently selected animal
    if multianimal==0:
        start_nan = n_frames-1
        for k in range(current_index[0], n_frames, 1):
            if np.isnan(tracesx[k,selected_animal[0]]):
                start_nan = k
                break
        print ("searching... start_nan: ", start_nan)

        start_nan+=1
        start_val = n_frames-1
        for k in range(start_nan, n_frames, 1):
            if np.isnan(tracesx[k,selected_animal[0]])==False:
                start_val=k
                break

        print ("Fast forwarding from: ", current_index[0])
        current_index[0] = start_val
        
        sfreq.val = current_index[0]
        #fig.canvas.draw_idle()

        print ("..... to : ", current_index[0])

        update(current_index[0])

    # Search all snippets for the next valid 
    else:
        print ("NOT IMPLENETED YET")
        pass
        start_val = n_frames-1
        for l in range(4):
            if l==selected_animal[0]:
                continue
            
            # search for first non nan value and animal id
            for k in range(current_index[0], n_frames, 1):
                if np.isnan(tracesx[k,l])==False:
                    if start_val>=k:
                        selected_animal[0] = l
                        start_val = k
                        break
            
                    
        print ("Fast forwarding from: ", current_index[0])
        current_index[0] = start_val
    
   
    
    
    # search for the next continuous chunk of red

    
def forward_function(event):
    
    # 
    current_index[0]+=1
    sfreq.val = current_index[0]

    update(current_index[0])
    #fig.canvas.draw_idle()
    
def backward_function(event):
    
    # 
    current_index[0]-=1
    sfreq.val = current_index[0]

    update(current_index[0])
    #fig.canvas.draw_idle()

        
def disconnect_jumps_function(event):
    
    global tracesx, tracesy
    # 
    print ("DELETING JUMPS FUCNTION")
    disconnect_jumps()

    update(current_index[0])
    #fig.canvas.draw_idle()
    
    
def previous_segment(event):
    
    # 
    print ("Current esgemtn: ", tracesx[current_index[0]:current_index[0]+5,
                                        selected_animal[0]])
    
    # END OF PREVIOUS NAN BLOCK
    end_nan=0
    for k in range(current_index[0], 0, -1):
        if np.isnan(tracesx[k,selected_animal[0]]):
            end_nan = k
            break
    print ("searching... start_nan: ", end_nan)

    # START OF PREVIOUS NAN BLOCK
    start_nan=0
    for k in range(end_nan, 0, -1):
        if np.isnan(tracesx[k,selected_animal[0]])==False:
            start_nan=k+1
            break

    start_data =0
    for k in range(start_nan-1, 0, -1):
        if np.isnan(tracesx[k,selected_animal[0]]):
            start_data=k+1 # select the very next val
            break
            
    print ("Reversing from: ", current_index[0])
    current_index[0] = start_data
    sfreq.val = current_index[0]

    print ("..... to : ", current_index[0])
    update(current_index[0])
    #fig.canvas.draw_idle()

    
def save(event):
    
    pattern = (('NPZ', '*.npz'),)
    title = 'Npz'
    
    traces_saved = filedialog.asksaveasfilename(filetypes=pattern, title=title)
    
    np.savez(traces_saved, 
             tracesx = tracesx,
             tracesy = tracesy)
    print ("SAVING TRACES")
        
def load_function(event):
    
    global tracesx, tracesy
    
    pattern = (('NPZ', '*.npz'),)
    title = 'Npz'

    traces_saved = filedialog.askopenfilename(filetypes=pattern, title=title)
    
    data = np.load(traces_saved)
    tracesx = data['tracesx']
    tracesy = data['tracesy']
    update(current_index[0])

    print ("loaded traces")

    
def interpolate_function(event):
    global tracesx, tracesy
    
    dist_threshold = 200.
    time_threshold = 25
    tracesx, tracesy = interpolate_traces(dist_threshold,
                                         time_threshold)
    
    update(current_index[0])
    #fig.canvas.draw_idle()
    print ("INTERPOLATING TRACES, THRESHOLD: ", dist_threshold)

# def interpolate_dynamic(event):
#     global tracesx, tracesy
    
#     dist_threshold = 200.
#     time_threshold = 25
#     tracesx, tracesy = interpolate_traces_dynamic(tracesx, tracesy, 
#                                           dist_threshold,
#                                          time_threshold)
    
#     update(current_index[0])
#     #fig.canvas.draw_idle()
#     print ("INTERPOLATING TRACES, THRESHOLD: ", dist_threshold)    
    
    
def export_movie(movie):
    clrs = ['blue','red','cyan','green']

    size_vid = np.array([1280,1024])
    dot_size = 16
    
    #out_dir = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/2020-3-9_08_18_49_128168/'
    fname_out = (out_dir+'video_labeled_exported_fixed.avi')
    fourcc = cv2.VideoWriter_fourcc('M','P','E','G')
    video_out = cv2.VideoWriter(fname_out,fourcc, 25, (size_vid[0],size_vid[1]), True)

    # initialize start and ends of video to be analyzed
    start = 0
    #end = start + 10*25
    end = tracesx.shape[0]
    #end = 3000
    original_vid = cv2.VideoCapture(video_name)
    original_vid.set(cv2.CAP_PROP_POS_FRAMES, start)

    for k in range(start,end,1):    
        if k %2500==0:
            print (k)

        ret, frame = original_vid.read()
        if ret==False:
            break
#         if scaling:
#             frame_cropped = frame.copy()[::4,::4]
#         else:
#             frame_cropped = frame.copy()

        # Loop over animals
        for p in range(0, tracesx.shape[1], 1):
            x = tracesx[k,p]
            y = tracesy[k,p]
            # plot large bump
            if np.isnan(x) or np.isnan(y):
                continue
            x = int(x)
            y = int(y)
            
            frame[y-dot_size:y+dot_size,x-dot_size:x+dot_size]= (np.float32(
                matplotlib.colors.to_rgb(clrs[p]))*255.).astype('uint8')

        video_out.write(frame)

    video_out.release()
    original_vid.release()
    cv2.destroyAllWindows()
    
    
    
    

In [3]:
# CELL #3
# change flag for windows
if False:
    linux = True


    if linux==True:
        out_dir = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/2020-3-9_08_18_49_128168/'
        video_name = os.path.join(out_dir, '2020-3-9_08_18_49_128168_compressed_snippet.avi')
        traces_means = os.path.join(out_dir, 'traces_means_unfiltered.npz') 
        traces_saved = os.path.join(out_dir,'traces_fixed.npz')
        %matplotlib tk

    else:
        #out_dir = r"C:\madeline\data\'
        video_name = r"C:\madeline\data\2020-3-9_08_18_49_128168_compressed_snippet.avi"
        traces_name = r"C:\madeline\data\traces_means.npz"
        traces_saved = r"C:\madeline\data\traces_saved.npz"
        %matplotlib qt


    print (video_name)
    data= np.load(traces_means)

    tracesx = data['tracesx_mean']
    tracesy = data['tracesy_mean']
    print ("DONE LOADING")
    
    
def load_dataset(event):
    
    global tracesx, tracesy, video_name, out_dir, original_vid, axes_animals, ax_tracesx, ax_tracesy, n_frames, img, t, init_length, sfreq
    
    
    pattern = (('AVI', '*.avi'),)
    title = 'Load .avi video to review'

    video_name = filedialog.askopenfilename(filetypes=pattern, title=title)
    
    out_dir = os.path.split(video_name)[0]
    
    
    # initialize video box
    original_vid = cv2.VideoCapture(video_name)
    #dlc.original_vid = original_vid


    # initialize video
    loc = 0
    original_vid.set(cv2.CAP_PROP_POS_FRAMES, loc)
    ret, frame = original_vid.read()
    n_frames = int(original_vid.get(cv2.CAP_PROP_FRAME_COUNT))

    
    #sfreq = Slider(axfreq, 'Frame', 0, n_frames, valinit=0, valstep=1, dragging=True)
    sfreq = Slider(axfreq, 'Frame', 0, n_frames, valinit=0, valstep=1, dragging=True)
    sfreq.on_changed(update)

    # initialize scatter plots
    ax_video = plt.subplot(gs[:, 1:4])
    img = ax_video.imshow(frame)
    ax_video.set_xlim(125,1225)
    ax_video.set_ylim(50,800)
    
    
    
    # load traces also
    pattern = (('NPZ', '*.npz'),)
    title = 'Load .npz file containing traces'

    traces_saved = filedialog.askopenfilename(filetypes=pattern, title=title)
    
    #traces_saved = video_name[:-4]+'_single_feature_traces.npz'
    #                               # _single_feature_traces
    #print ("fname: ", traces_saved)
    data = np.load(traces_saved)
    tracesx = data['tracesx']
    tracesy = data['tracesy']
    
    # initialize traces:
    axes_animals=[]
    for k in range(4):
        x_loc = tracesx[0,k]
        y_loc = tracesy[0,k]
        if np.isnan(x_loc) or np.isnan(y_loc):
            x_loc=100
            y_loc=100
        axes_animals.append(ax_video.scatter(x_loc, y_loc,c=clrs[k]))

    # plot traces
    ax_tracesx = plt.subplot(gs[:2, :1])
    ax_tracesx.set_title("DLC-x")
    init_length = 10*25
    t = np.arange(init_length)/25.
    ax1 = []
    for k in range(4):
        temp, = ax_tracesx.plot(t,tracesx[:init_length,k],c=clrs[k])
        ax1.append(temp)

    ax_tracesx.set_xticks([])
    ax_tracesx.set_ylim(0,1280)

    # plot y traces
    ax_tracesy = plt.subplot(gs[2:, :1])
    ax_tracesy.set_title("DLC-y")
    ax2 = []
    for k in range(4):
        temp, = ax_tracesy.plot(t,tracesy[:init_length,k],c=clrs[k])
        ax2.append(temp)

    ax_tracesy.set_xlabel("Time (sec)")
    ax_tracesy.set_ylim(0,1024)

    # initialize scatter plot



    #video_name = os.path.join(out_dir, '2020-3-9_08_18_49_128168_compressed_snippet.avi')

    update(current_index[0])

    print ("loaded video")
    

In [10]:
# CELL 4 LOAD GUI
global current_index, selected_animal, tracesx, tracesy, traces_saved, n_frames, autoadvance, frame_rate, sfreq, original_vid, axes_animals, ax_tracesx, ax_tracesy, fig

# set plotting figure
fig = plt.figure()
gs = gridspec.GridSpec(4, 4)
gs.tight_layout(fig)

clrs = ['red','blue','cyan','green']
sizes = np.ones(4,'int32')*50
sizes[0] = 250

# auto advance default setting
autoadvance = 0

# multianimal setting: cycle through segments for all animals, not just specific animal
multianimal = 0

# save current frame of video
current_index = []
current_index.append(0)
#dlc.current_index=current_index

# save selected animal radio button 
selected_animal = []
selected_animal.append(0)

# initialize start and ends of video to be analyzed
frame_rate = 25

# ***************** SLIDER **********************
##location of sliders
axcolor = 'lightgoldenrodyellow'
axfreq = plt.axes([0.01, 0.01, 0.94, 0.03], facecolor=axcolor)
n_frames = 10
sfreq = Slider(axfreq, 'Frame', 0, n_frames, valinit=0, valstep=1, dragging=True)
sfreq.on_changed(update)

# ##################### BUTTONS #####################

# Auto advance SELECTOR
labels = ["OFF", "ON"]
rax0 = plt.axes([0.01, 0.90, 0.04, 0.05], facecolor=axcolor)
advance_buttons = RadioButtons(rax0, (labels[0],labels[1]), active=0)
rax0.set_title('Auto\nadvance')
advance_buttons.on_clicked(advance_function)
for circle in advance_buttons.circles: # adjust radius here. The default is 0.05
    circle.set_radius(0.1)

#labels = ["OFF", "ON"]
rax00 = plt.axes([0.06, 0.90, 0.04, 0.05], facecolor=axcolor)
multi_animal = RadioButtons(rax00, (labels[0],labels[1]), active=0)
rax00.set_title('Multi\nAnimal')
multi_animal.on_clicked(multi_animal_skip)
for circle in multi_animal.circles: # adjust radius here. The default is 0.05
    circle.set_radius(0.1)


# ANIMAL SELECTOR
labels = ["female (red)", "male (blue)", "pup1 (cyan)", "pup2 (green)"]
rax1 = plt.axes([0.01, 0.80, 0.07, 0.08], facecolor=axcolor)
animal_buttons = RadioButtons(rax1, (labels[0],labels[1],labels[2],labels[3]), active=0)
rax1.set_title('Select')
animal_buttons.on_clicked(animal_select)
for circle in animal_buttons.circles: # adjust radius here. The default is 0.05
    circle.set_radius(0.1)
    
# ANIMAL SWITCHER
rax2 = plt.axes([0.01, 0.70, 0.07, 0.08], facecolor=axcolor)
animal_switch = RadioButtons(rax2, (labels[0],labels[1],labels[2],labels[3]), active=0)
rax2.set_title('Switch') 
animal_switch.on_clicked(animal_switch_function)
for circle in animal_switch.circles: # adjust radius here. The default is 0.05
    circle.set_radius(0.1)
    
#axprev = plt.axes([0.7, 0.05, 0.1, 0.075])
axnext = plt.axes([0.01, 0.65, 0.04, 0.04])
bnext = Button(axnext, 'Next\nSegment')
bnext.on_clicked(next_segment)
#bprev = Button(axprev, 'Previous')
#bprev.on_clicked(callback.prev)

# Del Segment
axdel_seg = plt.axes([0.06, 0.65, 0.04, 0.04])
bdel_seg = Button(axdel_seg, 'Del\nSegment')
bdel_seg.on_clicked(del_segment)



#axprev = plt.axes([0.7, 0.05, 0.1, 0.075])
axprev = plt.axes([0.01, 0.60, 0.04, 0.04])
bprev = Button(axprev, 'Previous\nSegment')
bprev.on_clicked(previous_segment)
#bprev = Button(axprev, 'Previous')
#bprev.on_clicked(callback.prev)




#axprev = plt.axes([0.7, 0.05, 0.1, 0.075])
axforward = plt.axes([0.01, 0.50, 0.04, 0.04])
bforward = Button(axforward, 'Step\nForward')
bforward.on_clicked(forward_function)

# BACK BUTTON
axback = plt.axes([0.06, 0.50, 0.04, 0.04])
bback = Button(axback, 'Step\nBackward')
bback.on_clicked(backward_function)
#bprev = Button(axprev, 'Previous')




# INTERPOLATE
axnans = plt.axes([0.01, 0.40, 0.04, 0.04])
bnans = Button(axnans, 'Clean\nData',  color='0.85', hovercolor='0.95')
bnans.on_clicked(clean_data)
bnans.color='grey'
bnans.hovercolor='green'


# DELETE JUMPS BUTTON
axdel_jumps = plt.axes([0.01, 0.35, 0.04, 0.04])
bdeljumps = Button(axdel_jumps, 'Disconnect\nJumps')
bdeljumps.on_clicked(disconnect_jumps_function)
#bprev = Button(axprev, 'Previous')

# INTERPOLATE
axinterpolate = plt.axes([0.06, 0.35, 0.04, 0.04])
binterpoloate = Button(axinterpolate, 'Join\nTraces')
binterpoloate.on_clicked(interpolate_function)
#bprev = Button(axprev, 'Previous')

# # INTERPOLATE
# axjoin_dynamic = plt.axes([0.06, 0.25, 0.04, 0.04])
# bjoin_dynamic = Button(axjoin_dynamic, 'Join\nDynamic')
# bjoin_dynamic.on_clicked(interpolate_dynamic)
# #bprev = Button(axprev, 'Previous')

# # Delete very short segments automatically
# axtriples = plt.axes([0.01, 0.30, 0.04, 0.04])
# btriples = Button(axtriples, 'DelShort\nSegments')
# btriples.on_clicked(del_short_segments)



#bprev = Button(axprev, 'Previous')

    
    
    
#axprev = plt.axes([0.7, 0.05, 0.1, 0.075])
axswitch_all = plt.axes([0.01, 0.20, 0.04, 0.04])
bswitch_all = Button(axswitch_all, 'Switch\nTraces (10sec)')
bswitch_all.on_clicked(switch_all_screen)
#bprev = Button(axprev, 'Previous')
#bprev.on_clicked(callback.prev)


# #axprev = plt.axes([0.7, 0.05, 0.1, 0.075])
# axswap = plt.axes([0.06, 0.20, 0.04, 0.04])
# bswap = Button(axswap, 'SWAP\nANIMALS')
# bswap.on_clicked(animal_swap)
# #bprev = Button(axprev, 'Previous')
# #bprev.on_clicked(callback.prev)


#bprev = Button(axprev, 'Previous')

# Export movie
axmovie = plt.axes([0.01, 0.05, 0.04, 0.04])
bmovie = Button(axmovie, 'Export\nMovie')
bmovie.on_clicked(export_movie)
#bprev = Button(axprev, 'Previous')


#axprev = plt.axes([0.7, 0.05, 0.1, 0.075])
axload_dataset = plt.axes([0.06, 0.15, 0.04, 0.04])
bload_dataset = Button(axload_dataset, 'Load\nDataset', color='0.85', hovercolor='0.95')
bload_dataset.color='grey'
bload_dataset.hovercolor='r'

#button = Button(tkWindow, text = 'Submit', bg='blue', fg='white')  

bload_dataset.on_clicked(load_dataset)


#axprev = plt.axes([0.7, 0.05, 0.1, 0.075])
axsave = plt.axes([0.06, 0.05, 0.04, 0.04])
bsave = Button(axsave, 'Save\nTraces')
bsave.on_clicked(save)

#axprev = plt.axes([0.7, 0.05, 0.1, 0.075])
axload = plt.axes([0.11, 0.05, 0.04, 0.04])
bload = Button(axload, 'Load\nTraces')
bload.on_clicked(load_function)


load_dataset(0)

plt.show()

current indx updated: [0]
loaded video
DELETING JUMPS FUCNTION
DONE disconnecting JUMPS
current indx updated: [0]
DELETING JUMPS FUCNTION
current indx updated: [0]
current indx updated: [0]
INTERPOLATING TRACES, THRESHOLD:  200.0
DELETING JUMPS FUCNTION
current indx updated: [0]
current indx updated: [0]
INTERPOLATING TRACES, THRESHOLD:  200.0
DELETING JUMPS FUCNTION
current indx updated: [0]
current indx updated: [0]
INTERPOLATING TRACES, THRESHOLD:  200.0
current indx updated: [6741]
current indx updated: [10099]
current indx updated: [11804]
current indx updated: [13095]
current indx updated: [13457]
current indx updated: [12682]
current indx updated: [11080]
current indx updated: [10770]
current indx updated: [10615]
current indx updated: [10512]
DELETING JUMPS FUCNTION
DONE disconnecting JUMPS
current indx updated: [10512]
DELETING JUMPS FUCNTION
DONE disconnecting JUMPS
current indx updated: [10512]
DELETING JUMPS FUCNTION
DONE disconnecting JUMPS
current indx updated: [10512]
DE

deleted small segs 0 70603
deleted small segs 0 70607
deleted small segs 0 70780
deleted small segs 0 70805
deleted small segs 0 70828
deleted small segs 0 71035
deleted small segs 0 71687
deleted small segs 0 71691
deleted small segs 0 72021
deleted small segs 0 72234
deleted small segs 0 72261
deleted small segs 0 72287
deleted small segs 0 72290
deleted small segs 0 72747
deleted small segs 0 73100
deleted small segs 0 73159
deleted small segs 0 73674
deleted small segs 0 73960
deleted small segs 0 75606
deleted small segs 0 75633
deleted small segs 0 76287
deleted small segs 0 78192
deleted small segs 0 78207
deleted small segs 0 78471
deleted small segs 0 79818
deleted small segs 0 79844
deleted small segs 0 79858
deleted small segs 0 81021
deleted small segs 0 81605
deleted small segs 0 81609
deleted small segs 0 81710
deleted small segs 0 81861
deleted small segs 0 81864
deleted small segs 0 81869
deleted small segs 0 81892
deleted small segs 0 82264
deleted small segs 0 82268
d

deleted small segs 3 57031
deleted small segs 3 57145
deleted small segs 3 57351
deleted small segs 3 57381
deleted small segs 3 57572
deleted small segs 3 57584
deleted small segs 3 61574
deleted small segs 3 66785
deleted small segs 3 66970
deleted small segs 3 70541
deleted small segs 3 83125
DONE DELEting triples
current indx updated: [28450]
DONE Deleting short segments
current indx updated: [28450]
searching... start_nan:  28450
Fast forwarding from:  28450
..... to :  28460
current indx updated: [28460]
searching... start_nan:  28468
Fast forwarding from:  28460
..... to :  28472
current indx updated: [28472]
searching... start_nan:  28519
Fast forwarding from:  28472
..... to :  28634
current indx updated: [28634]
DELETING JUMPS FUCNTION
current indx updated: [28634]
current indx updated: [28634]
INTERPOLATING TRACES, THRESHOLD:  200.0
DELETING JUMPS FUCNTION
current indx updated: [28634]
current indx updated: [28634]
INTERPOLATING TRACES, THRESHOLD:  200.0
DELETING JUMPS FUCNT

searching... start_nan:  57829
Fast forwarding from:  57713
..... to :  57831
current indx updated: [57831]
searching... start_nan:  57879
Fast forwarding from:  57831
..... to :  57913
current indx updated: [57913]
searching... start_nan:  58057
Fast forwarding from:  57913
..... to :  58059
current indx updated: [58059]
searching... start_nan:  58084
Fast forwarding from:  58059
..... to :  58086
current indx updated: [58086]
searching... start_nan:  58764
Fast forwarding from:  58086
..... to :  58790
current indx updated: [58790]
current indx updated: [5449]
current indx updated: [0]
current indx updated: [1109]
current indx updated: [1058]
current indx updated: [593]
current indx updated: [0]
searching... start_nan:  145
Fast forwarding from:  0
..... to :  818
current indx updated: [818]
searching... start_nan:  1505
Fast forwarding from:  818
..... to :  1890
current indx updated: [1890]
searching... start_nan:  1902
Fast forwarding from:  1890
..... to :  7949
current indx upda

searching... start_nan:  23910
Fast forwarding from:  23838
..... to :  23912
current indx updated: [23912]
searching... start_nan:  23917
Fast forwarding from:  23912
..... to :  23919
current indx updated: [23919]
Current esgemtn:  [239.6985 239.6985 267.311  267.4275 248.8425]
searching... start_nan:  23918
Reversing from:  23919
..... to :  23912
current indx updated: [23912]
Selected animal id:  [0] , start/end:  23912 23917
DONE DELEting triples
current indx updated: [23912]
searching... start_nan:  23912
Fast forwarding from:  23912
..... to :  23919
current indx updated: [23919]
searching... start_nan:  24013
Fast forwarding from:  23919
..... to :  24015
current indx updated: [24015]
Selected animal id:  [0] , start/end:  24015 24022
DONE DELEting triples
current indx updated: [24015]
searching... start_nan:  24015
Fast forwarding from:  24015
..... to :  24024
current indx updated: [24024]
searching... start_nan:  24313
Fast forwarding from:  24024
..... to :  24315
current i



SWITCH FUNCTION
switching to :  female (red)
current index inside switch: [31633]
Switch animal id:  0 , start/end:  30368 32413
selected animal id:  1 , start/end:  31633 31637
current indx updated: [31633]
searching... start_nan:  32413
Fast forwarding from:  31633
..... to :  32483
current indx updated: [32483]
Current esgemtn:  [276.375  287.25   299.559  332.0315 335.298 ]
searching... start_nan:  32482
Reversing from:  32483
..... to :  31633
current indx updated: [31633]
current indx updated: [31633]
searching... start_nan:  31637
Fast forwarding from:  31633
..... to :  31639
current indx updated: [31639]
searching... start_nan:  31693
Fast forwarding from:  31639
..... to :  31728
current indx updated: [31728]
searching... start_nan:  31735
Fast forwarding from:  31728
..... to :  31862
current indx updated: [31862]
DELETING JUMPS FUCNTION
DONE disconnecting JUMPS
current indx updated: [31862]
searching... start_nan:  31931
Fast forwarding from:  31862
..... to :  31933
curr

DELETING JUMPS FUCNTION
DONE disconnecting JUMPS
current indx updated: [43293]
searching... start_nan:  43293
Fast forwarding from:  43293
..... to :  43320
current indx updated: [43320]
searching... start_nan:  43321
Fast forwarding from:  43320
..... to :  43323
current indx updated: [43323]


SWITCH FUNCTION
switching to :  pup2 (green)
current index inside switch: [43323]
Switch animal id:  3 , start/end:  43322 43334
selected animal id:  0 , start/end:  43323 43330
current indx updated: [43323]
searching... start_nan:  43334
Fast forwarding from:  43323
..... to :  43339
current indx updated: [43339]
searching... start_nan:  43350
Fast forwarding from:  43339
..... to :  43360
current indx updated: [43360]
Selected animal id:  [0] , start/end:  43360 43364
DONE DELEting triples
current indx updated: [43360]
searching... start_nan:  43360
Fast forwarding from:  43360
..... to :  43366
current indx updated: [43366]


SWITCH FUNCTION
switching to :  pup2 (green)
current index inside 



SWITCH FUNCTION
switching to :  pup2 (green)
current index inside switch: [46833]
Switch animal id:  3 , start/end:  46834 46833
selected animal id:  0 , start/end:  46833 46842
current indx updated: [46833]
searching... start_nan:  46833
Fast forwarding from:  46833
..... to :  46844
current indx updated: [46844]


SWITCH FUNCTION
switching to :  pup2 (green)
current index inside switch: [46844]
Switch animal id:  3 , start/end:  46845 46844
selected animal id:  0 , start/end:  46844 47019
current indx updated: [46844]
DELETING JUMPS FUCNTION
DONE disconnecting JUMPS
current indx updated: [46844]
searching... start_nan:  46844
Fast forwarding from:  46844
..... to :  46881
current indx updated: [46881]
searching... start_nan:  46891
Fast forwarding from:  46881
..... to :  46896
current indx updated: [46896]
searching... start_nan:  47019
Fast forwarding from:  46896
..... to :  47021
current indx updated: [47021]
searching... start_nan:  47205
Fast forwarding from:  47021
..... to 

searching... start_nan:  54808
Fast forwarding from:  54711
..... to :  54810
current indx updated: [54810]
searching... start_nan:  54928
Fast forwarding from:  54810
..... to :  54955
current indx updated: [54955]


SWITCH FUNCTION
switching to :  male (blue)
current index inside switch: [54955]
Switch animal id:  1 , start/end:  54322 54999
selected animal id:  0 , start/end:  54955 54973
current indx updated: [54955]
searching... start_nan:  54999
Fast forwarding from:  54955
..... to :  55110
current indx updated: [55110]


SWITCH FUNCTION
switching to :  pup1 (cyan)
current index inside switch: [55110]
Switch animal id:  2 , start/end:  55111 55110
selected animal id:  0 , start/end:  55110 55136
current indx updated: [55110]
current indx updated: [55110]


SWITCH FUNCTION
switching to :  female (red)
current index inside switch: [55110]
Switch animal id:  0 , start/end:  55111 55110
selected animal id:  1 , start/end:  55001 55171
current indx updated: [55110]
DELETING JUMPS FUC

searching... start_nan:  61352
Fast forwarding from:  60350
..... to :  61354
current indx updated: [61354]
searching... start_nan:  61359
Fast forwarding from:  61354
..... to :  61515
current indx updated: [61515]


SWITCH FUNCTION
switching to :  pup1 (cyan)
current index inside switch: [61515]
Switch animal id:  2 , start/end:  61512 61576
selected animal id:  0 , start/end:  61515 61530
current indx updated: [61515]


SWITCH FUNCTION
switching to :  pup2 (green)
current index inside switch: [61515]
Switch animal id:  3 , start/end:  61516 61515
selected animal id:  0 , start/end:  61515 61576
current indx updated: [61515]
searching... start_nan:  61515
Fast forwarding from:  61515
..... to :  61579
current indx updated: [61579]


SWITCH FUNCTION
switching to :  pup1 (cyan)
current index inside switch: [61579]
Switch animal id:  2 , start/end:  61580 61579
selected animal id:  0 , start/end:  61579 61657
current indx updated: [61579]


In [22]:
# CELL 5 : visualize results
animal_id = 0
t=np.arange(tracesx.shape[0])/25.

lens = []
for k in range(4):
    print (k)
    ax=plt.subplot(4,1,k+1)
    temp = tracesx[:,k]+tracesy[:,k]
    plt.plot(t,temp)
    idx = np.isnan(temp)
    print (idx.shape)
    idx2 = np.where(idx==False)[0]

    #print (idx2.shape)
    #lens.append(idx2.shape[0])
    #plt.plot(t[idx2],temp[idx2],c='red')
    plt.title("Animal: "+ labels[k]+ " , labeled frames: " +str(np.round(idx2.shape[0]/tracesx.shape[0]*100, 2)) +"%",fontsize=16)
    
    plt.xlim(t[0],t[-1])
    if k!=3:
        plt.xticks([])
    else:
        plt.xlabel("Time (sec)",fontsize=20)
        
print (np.array(lens)/16572)
#for k in range(4):
#    ax=plt.subplot(4,1,k+1)
#    plt.plot(t,tracesy[:,k])

    
plt.show()

0
(89989,)
1
(89989,)
2
(89989,)
3
(89989,)
[]


In [ ]:
# FUTURE WORK OBJECT ORIENTED PROGRAMMING

# reload data
#tracesx = tracesx_ip.copy() #data['tracesx_mean'].copy()
#tracesy = tracesy_ip.copy() #data['tracesy_mean'].copy()

# class dlc_object():

#     def __init__(self, tracesx, tracesy, traces_saved):
#         self.tracesx=tracesx.copy()
#         self.tracesy=tracesy.copy()        
#         self.traces_saved=traces_saved
        
#         self.current_index = []
#         self.selected_animal = []
#         self.original_vid = []
#         self.n_frames = []

        
        
# dlc = dlc_object(tracesx, tracesy,
#                 traces_saved)
# print (dlc.tracesx.shape)


In [2]:
# id_=2
# for k in range(tracesx.shape[0]):
#     if np.isnan(tracesx[k,id_])!=np.isnan(tracesy[k,id_]):
#         print (k, tracesx[k,id_], tracesy[k,id_])

NameError: name 'tracesx' is not defined